In [ ]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')
data

: 

In [ ]:
timeseries = df[["Passengers"]].values.astype('float32')
 
plt.plot(timeseries)
plt.show()